In [13]:
import pandas as pd
df= pd.read_csv("Emotions.csv")
import nltk
from nltk.corpus import stopwords
nltk.download('stopwords')
stop_words = set(stopwords.words("english"))


[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Lenovo\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [14]:
def remove_stopwords(text):
    new_text=[]
    for word in text.split():
        if word in stop_words:
            new_text.append('')
            
        else:
            new_text.append(word)
            
    x= new_text[:]
    new_text.clear()
    return ' '.join(x)

In [15]:
df1= df.loc[0:5000]

In [16]:
df1['text']=df1['text'].apply(remove_stopwords)

C:\Users\Lenovo\AppData\Local\Temp\ipykernel_3616\280909826.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df1['text']=df1['text'].apply(remove_stopwords)


In [18]:
from nltk.stem import WordNetLemmatizer
nltk.download("wordnet")
nltk.download("omw-1.4")
wnl = WordNetLemmatizer()
def lemmatiza_wrds(text):
    new_text=[]
    for word in text.split():
        le_wrd= wnl.lemmatize(word, pos="v")
        new_text.append(le_wrd)
        if (word != le_wrd):
            print(word, le_wrd)
        
            
    x= new_text[:]
    new_text.clear()
    return ' '.join(x)

[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\Lenovo\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to
[nltk_data]     C:\Users\Lenovo\AppData\Roaming\nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


In [20]:
from sklearn.model_selection import train_test_split
x= df1.iloc[:,1:2]
y= df1['label']
X_train,X_test,y_train,y_test= train_test_split(x,y,test_size=0.2,random_state=1)

In [21]:
from sklearn.feature_extraction.text import TfidfVectorizer
tfvd= TfidfVectorizer(min_df=0.0, max_df=1.0, ngram_range=(1,2),sublinear_tf=True)
tfv_train= tfvd.fit_transform(X_train['text']).toarray()
tfv_test= tfvd.transform(X_test['text']).toarray()

In [28]:
from sklearn.feature_extraction.text import CountVectorizer
cv= CountVectorizer()
x_train_cv= cv.fit_transform(X_train['text']).toarray()
x_test_cv= cv.transform(X_test['text']).toarray()

# RF with tfidf

In [23]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score ,confusion_matrix , classification_report
rf= RandomForestClassifier(n_estimators=50)
rf.fit(tfv_train,y_train)
y_pred= rf.predict(tfv_test)
accuracy_score(y_test,y_pred)
print(classification_report(y_test, y_pred))
print(confusion_matrix(y_test, y_pred))      

              precision    recall  f1-score   support

           0       0.87      0.88      0.87       295
           1       0.77      0.92      0.84       329
           2       0.87      0.59      0.71        79
           3       0.88      0.77      0.82       145
           4       0.90      0.71      0.80       119
           5       0.79      0.79      0.79        34

    accuracy                           0.83      1001
   macro avg       0.85      0.78      0.81      1001
weighted avg       0.84      0.83      0.83      1001

[[259  26   3   6   1   0]
 [ 16 304   2   2   3   2]
 [  5  25  47   2   0   0]
 [  8  21   1 112   3   0]
 [  8  14   1   6  85   5]
 [  2   3   0   0   2  27]]


# RF with CV

In [29]:
rf= RandomForestClassifier(n_estimators=50)
rf.fit(x_train_cv,y_train)
y_pred= rf.predict(x_test_cv)
print(accuracy_score(y_test,y_pred))
print(classification_report(y_test, y_pred))
print(confusion_matrix(y_test, y_pred)) 

0.8631368631368631
              precision    recall  f1-score   support

           0       0.93      0.90      0.92       295
           1       0.83      0.91      0.87       329
           2       0.78      0.68      0.73        79
           3       0.88      0.81      0.84       145
           4       0.87      0.82      0.84       119
           5       0.73      0.88      0.80        34

    accuracy                           0.86      1001
   macro avg       0.84      0.83      0.83      1001
weighted avg       0.86      0.86      0.86      1001

[[266  15   4   6   3   1]
 [  8 300   9   2   6   4]
 [  2  20  54   2   1   0]
 [  6  18   1 117   3   0]
 [  4   5   1   6  97   6]
 [  0   2   0   0   2  30]]


# Dt with tfidf

In [25]:
from sklearn.tree import DecisionTreeClassifier
classifier_tree = DecisionTreeClassifier()
y_pred = classifier_tree.fit(tfv_train, y_train).predict(tfv_test)
print(accuracy_score(y_test,y_pred))
print(classification_report(y_test, y_pred))
print(confusion_matrix(y_test, y_pred))

0.8621378621378621
              precision    recall  f1-score   support

           0       0.92      0.91      0.91       295
           1       0.89      0.85      0.87       329
           2       0.74      0.84      0.79        79
           3       0.83      0.87      0.85       145
           4       0.83      0.80      0.82       119
           5       0.67      0.85      0.75        34

    accuracy                           0.86      1001
   macro avg       0.82      0.85      0.83      1001
weighted avg       0.87      0.86      0.86      1001

[[268  10   4   8   4   1]
 [ 10 279  18   7   9   6]
 [  2   7  66   3   1   0]
 [  4  12   1 126   2   0]
 [  7   3   0   7  95   7]
 [  0   2   0   0   3  29]]


# DT with Cv: Best model

In [30]:
classifier_tree = DecisionTreeClassifier()
y_pred = classifier_tree.fit(x_train_cv, y_train).predict(x_test_cv)
print(accuracy_score(y_test,y_pred))
print(classification_report(y_test, y_pred))
print(confusion_matrix(y_test, y_pred)) 

0.8711288711288712
              precision    recall  f1-score   support

           0       0.92      0.91      0.92       295
           1       0.90      0.86      0.88       329
           2       0.73      0.80      0.76        79
           3       0.85      0.86      0.86       145
           4       0.83      0.86      0.84       119
           5       0.75      0.88      0.81        34

    accuracy                           0.87      1001
   macro avg       0.83      0.86      0.85      1001
weighted avg       0.87      0.87      0.87      1001

[[268   7   4  10   5   1]
 [ 11 284  18   4   8   4]
 [  3   9  63   3   1   0]
 [  3  12   1 125   4   0]
 [  5   2   0   5 102   5]
 [  0   1   0   0   3  30]]


# Log R with tfidf

In [26]:
from sklearn.linear_model import LogisticRegression
clf = LogisticRegression(random_state=0)
y_pred = clf.fit(tfv_train, y_train).predict(tfv_test)
print(accuracy_score(y_test,y_pred))
print(classification_report(y_test, y_pred))
print(confusion_matrix(y_test, y_pred))


0.7172827172827173
              precision    recall  f1-score   support

           0       0.76      0.93      0.83       295
           1       0.64      0.97      0.77       329
           2       1.00      0.13      0.22        79
           3       0.89      0.46      0.61       145
           4       0.90      0.39      0.55       119
           5       0.33      0.03      0.05        34

    accuracy                           0.72      1001
   macro avg       0.75      0.49      0.51      1001
weighted avg       0.76      0.72      0.67      1001

[[275  19   0   1   0   0]
 [ 11 318   0   0   0   0]
 [ 15  54  10   0   0   0]
 [ 28  49   0  67   1   0]
 [ 27  36   0   7  47   2]
 [  8  21   0   0   4   1]]


# Log R with CV

In [31]:
y_pred = clf.fit(x_train_cv, y_train).predict(x_test_cv)
print(accuracy_score(y_test,y_pred))
print(classification_report(y_test, y_pred))
print(confusion_matrix(y_test, y_pred))

0.8621378621378621
              precision    recall  f1-score   support

           0       0.88      0.93      0.91       295
           1       0.84      0.95      0.89       329
           2       0.83      0.62      0.71        79
           3       0.86      0.82      0.84       145
           4       0.89      0.77      0.83       119
           5       0.85      0.50      0.63        34

    accuracy                           0.86      1001
   macro avg       0.86      0.77      0.80      1001
weighted avg       0.86      0.86      0.86      1001

[[275   9   3   8   0   0]
 [  8 311   6   2   1   1]
 [  5  21  49   2   1   1]
 [ 12  10   0 119   4   0]
 [  9  10   0   7  92   1]
 [  3   8   1   0   5  17]]


C:\Users\Lenovo\AppData\Local\anaconda3\Lib\site-packages\sklearn\linear_model\_logistic.py:460: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


# Naive Bayes with tfidf

In [27]:
from sklearn.naive_bayes import GaussianNB  
classifier = GaussianNB() 
y_pred = classifier.fit(tfv_train, y_train).predict(tfv_test)
print(accuracy_score(y_test,y_pred))
print(classification_report(y_test, y_pred))
print(confusion_matrix(y_test, y_pred))


0.6093906093906094
              precision    recall  f1-score   support

           0       0.68      0.73      0.70       295
           1       0.61      0.75      0.67       329
           2       0.47      0.25      0.33        79
           3       0.60      0.50      0.54       145
           4       0.53      0.45      0.49       119
           5       0.25      0.12      0.16        34

    accuracy                           0.61      1001
   macro avg       0.52      0.47      0.48      1001
weighted avg       0.59      0.61      0.60      1001

[[214  38   7  19  15   2]
 [ 38 246  12  14  17   2]
 [  9  45  20   2   2   1]
 [ 23  37   2  72  10   1]
 [ 24  25   2   8  54   6]
 [  8  14   0   5   3   4]]


# Naive bayes with CV 

In [32]:
y_pred = classifier.fit(x_train_cv, y_train).predict(x_test_cv)
print(accuracy_score(y_test,y_pred))
print(classification_report(y_test, y_pred))
print(confusion_matrix(y_test, y_pred))

0.4805194805194805
              precision    recall  f1-score   support

           0       0.56      0.52      0.54       295
           1       0.61      0.52      0.56       329
           2       0.28      0.29      0.29        79
           3       0.44      0.46      0.45       145
           4       0.36      0.55      0.43       119
           5       0.08      0.09      0.09        34

    accuracy                           0.48      1001
   macro avg       0.39      0.40      0.39      1001
weighted avg       0.50      0.48      0.49      1001

[[153  28  18  35  48  13]
 [ 55 170  28  32  36   8]
 [ 16  33  23   3   3   1]
 [ 19  24   9  67  23   3]
 [ 23  10   3  10  65   8]
 [  6  13   0   6   6   3]]
